In [ ]:
import importlib
import functions
importlib.reload(functions)
from functions import *
directory_check()

In [ ]:
# This snippet widens the jupyter notebook layout for coding
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# We check and initialize all missing pre-process folder
directory_check()

In [ ]:
# Files and image data extraction
df_files = df_files_update(df_files)

In [ ]:
# Pre-Processing Files
df_files = df_files_preprocess(df_files)

In [ ]:
# Interface Detection and Extraction
df_files = df_files_interface(df_files)

In [ ]:
#Threshold Detection and Save
df_files = df_files_threshold(df_files)

In [ ]:
df_files_check(df_files)

In [ ]:
display(df_files.head())

In [ ]:
# Slicing Operation - Random
df_slice = df_slice_update(df_files, df_slice, slice_sq_dim = 224, overlap = "Yes", no_of_slices = 20)
# df_slice_intermediate = df_slice_file_display()
df_slice_check(df_slice)

In [ ]:
# Porosity Extraction
df_slice_porosity = df_slice_porosity_update(df_slice)

In [ ]:
# Porosity Updation till the last processed slice
df_slice_porosity = df_slice_porosity_file_display()

In [ ]:
df_slice_porosity.head()
df_slice_porosity.Sample.unique()

In [ ]:
# Mercury Porosimetry Data Extraction
datas_, Hg_df_dict = df_mercury_data_extract()

In [ ]:
for key, value in Hg_df_dict.items():
    print(key)

In [ ]:
# Convert Mercury Porosimetry Data Points to Important Sampling Points
Hg_porosity_df = df_mercury_common_cordinates(datas_, Hg_df_dict, df_slice_porosity, num = 10, kind = "linear") 

In [ ]:
display(Hg_porosity_df)

In [ ]:
# Coating parameter extraction
df_parameters = df_parameters_data_extract()

In [ ]:
df_parameters

In [ ]:
# Coating parameter index segmentation
df_slice_parameters = df_parameters_match(df_parameters, df_slice_porosity, bounded = True, max_powder_dia = 200, div = 10)
# display(df_slice_parameters.head())

In [ ]:
df_slice_parameters.head()

In [ ]:
# Coating roughness extraction                   
df_roughness = df_roughness_data_extract()
# display(df_roughness)

In [ ]:
df_roughness

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape, BatchNormalization, MaxPool2D, Convolution2D, InputLayer, Flatten, Concatenate, Lambda, Activation
from keras.utils import plot_model
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras import backend as K

def Cumulative_Sum(inp):
    return K.reverse(K.cumsum(inp,axis=1),axes=1)

# import tensorflow as tf
# config = tf.compat.v1.ConfigProto
# session = tf.compat.v1.InteractiveSession

# 2000X Slice Images generated "on_the_Go" = 2000_image_only_model

In [ ]:
# Making dataframe of 2000X variables (locatoions) and labels
df_files_2000_temp = (df_files.loc[df_files['Magnification'] == 2000.0])[['Sample','Image_Location_PostPocess']]
Hg_porosity_df_unique = Hg_porosity_df[['Sample', 'log_Radii_new', 'Ps_new']]

Hg_porosity_df_unique = Hg_porosity_df_unique.sort_values('Sample', ascending=True)
Hg_porosity_df_unique = Hg_porosity_df_unique.drop_duplicates(subset='Sample', keep='first')
Hg_porosity_df_unique = Hg_porosity_df_unique.reset_index(drop=True)
df_files_2000 = df_files_2000_temp[['Sample', 'Image_Location_PostPocess']].merge(Hg_porosity_df_unique, 
                                                                                  left_on='Sample', right_on='Sample')
Y_Train_Hg_2000 = np.array(df_files_2000["Ps_new"].tolist())
Y_Train_Hg_2000 = Y_Train_Hg_2000 / 100.
df_2000_train, df_2000_holdout, Y_Train_Hg_2000_train, Y_Train_Hg_2000_holdout = train_test_split(df_files_2000, Y_Train_Hg_2000, 
                                                                                                shuffle = True, random_state = 12, test_size = 0.1)
X_Train__img_2000_loc_train = np.array(df_2000_train["Image_Location_PostPocess"].tolist(), dtype = 'U')
X_Train__img_2000_train = []
for loc in X_Train__img_2000_loc_train:
    img = cv2.imread(str(loc), cv2.IMREAD_UNCHANGED) 
    X_Train__img_2000_train.append(img)

In [ ]:
def build_2000_image_only_model(nfilter = 16, filter_size = 3):   
    model1 = Sequential()
    model1.add(InputLayer(input_shape=(224,224), name= "bulk_img_input_2000"))
    model1.add(Reshape((224,224,1)))
    model1.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model1.add(Activation("relu"))
    model1.add(MaxPool2D())
    model1.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model1.add(Activation("relu"))
    model1.add(MaxPool2D())
    model1.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model1.add(Activation("relu"))
    model1.add(MaxPool2D())
    model1.add(Flatten())
    
    
    dense_1 = Dense(100, activation="tanh")(model1.output)
    dense_2 = Dense(10, activation="relu",name= "Hg_porosity_diff")(dense_1)
    lambda_layer = Lambda(Cumulative_Sum,name= "Hg_porosity")(dense_2)    
    big_model = Model([model1.input],[lambda_layer])

    big_model.compile("adam",loss="mean_squared_error")
    return big_model   

In [ ]:
def custom_generator(img_files, Hg_files, img_diam = 224, num_samples = 100, seed = None):
    i = 0
    tot_num_images = np.shape(img_files)[0]
    threshold_image = []
    hg_porosity = []
    
    X_train = []
    X_holdout = []
    Y_Train = []
    Y_holdout = []
    
    if seed is not None:
        np.random.seed(seed)
        
    rand_list = [np.random.randint(0,tot_num_images) for i in range(num_samples)]
    # Random.randint
    
    while i<num_samples:
        print(i)
        rand_index = rand_list[i]
        img = img_to_slices(img_files[rand_index], diam = 224, no_of_slices = 1)
        view2_image(img_files[rand_index], img[0])
        threshold_image.append(img[0])
        hg_porosity.append(Hg_files[rand_index])
        i = i + 1
        
    return threshold_image, hg_porosity

In [ ]:
X_train, Y_Train = custom_generator(X_Train__img_2000_train, Y_Train_Hg_2000_train,
                                                          img_diam = 224, num_samples = 10, seed = 3)

In [ ]:
X_train = np.array(X_train)/255
X_holdout = np.array(X_holdout)/255

In [ ]:
image_only_model_2000 = build_2000_image_only_model(nfilter = 4, filter_size=3)
history_total = image_only_model_2000.fit(x = [X_train], y = [Y_Train], batch_size=20,
                                          validation_split=0.15, epochs=50,verbose=1)

In [ ]:
plt.plot(history_total.history["loss"],label="train")
plt.plot(history_total.history["val_loss"],label="test")
plt.legend()
plt.yscale('log')

In [ ]:
Y_pred = image_only_model_2000.predict([X_holdout]) 

In [ ]:
for y_pred_i, y_true_i in zip(Y_pred, Y_holdout):
    plt.plot(y_pred_i, label="pred")
    plt.plot(y_true_i,label="True")
    plt.legend()
    plt.show()

In [ ]:
print(Y_pred)

# 500X Images and ML combinations using data from database

In [ ]:
df_total = pd.merge(df_slice, df_slice_porosity)
Y_Train_Hg_porosity = np.array(Hg_porosity_df["Ps_new"].tolist())
Y_Train_Hg_porosity = Y_Train_Hg_porosity / 100.
df_total["Points_area"] = df_total["Points_area"]/df_total["Total_area"]  
df_total["Threads_area"] = df_total["Threads_area"]/df_total["Total_area"]  
df_total["Clusters_area"] = df_total["Clusters_area"]/df_total["Total_area"]  
df_total["Slice_Array"] = df_total["Slice_Array"].map(lambda l:np.array(l)/255)
df_total["Interface_Array"] = df_total["Interface_Array"].map(lambda l:np.array(l)/255)
df_total["Slice_Array_Threshold"] = df_total["Slice_Array_Threshold"].map(lambda l:np.array(l)/255)

In [ ]:
display(df_total)

In [ ]:
df_train, df_holdout, Y_Train_Hg_porosity_train, Y_Train_Hg_porosity_holdout = train_test_split(df_total, Y_Train_Hg_porosity, shuffle = True, random_state = 12, test_size = 0.1)

In [ ]:
assert Y_Train_Hg_porosity.shape[0] == Y_Train_Hg_porosity_train.shape[0] + Y_Train_Hg_porosity_holdout.shape[0]
assert df_total.shape[0] ==df_train.shape[0] + df_holdout.shape[0]
assert Y_Train_Hg_porosity_train.shape[0] == df_train.shape[0]
assert df_holdout.shape[0] == Y_Train_Hg_porosity_holdout.shape[0]

X_Train_bulk_img_train = np.array(df_train["Slice_Array"].tolist())
X_Train_inter_img_train =  np.array(df_train["Interface_Array"].tolist())
X_Train_porosity_train =df_train[["Points_area","Threads_area","Clusters_area"]].values

X_Train_bulk_img_holdout = np.array(df_holdout["Slice_Array"].tolist())
X_Train_inter_img_hodlout =  np.array(df_holdout["Interface_Array"].tolist())
X_Train_porosity_holdout =df_holdout[["Points_area","Threads_area","Clusters_area"]].values

In [ ]:
def build_image_only_model(nfilter = 16, filter_size = 3):
    model = Sequential()
    model.add(InputLayer(input_shape=(224,224), name="bulk_img_input"))
    model.add(Reshape((224,224,1)))
    model.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Flatten())

    model2 = Sequential()
    model2.add(InputLayer(input_shape=(224, 224), name="interface_img_input"))
    model2.add(Reshape((224,224,1)))
    model2.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model2.add(BatchNormalization())
    model2.add(Activation("relu"))
    model2.add(MaxPool2D())
    
    model2.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model2.add(BatchNormalization())
    model2.add(Activation("relu"))
    model2.add(MaxPool2D())
    
    model2.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model2.add(BatchNormalization())
    model2.add(Activation("relu"))
    model2.add(MaxPool2D())
    
    model2.add(Flatten())
   
    concat_layer = Concatenate()([model.output,model2.output])

    dense_1 = Dense(100, activation="tanh")(concat_layer)
    dense_2 = Dense(10, activation="relu",name= "Hg_porosity_diff")(dense_1)
    lambda_layer = Lambda(Cumulative_Sum,name= "Hg_porosity")(dense_2)    
    big_model = Model([model.input, model2.input],[lambda_layer])
    #big_model = Model([model.input, model2.input],[dense_2])

    big_model.compile("adam",loss="mean_squared_error")
    return big_model

In [ ]:
image_only_model = build_image_only_model(nfilter = 4, filter_size=3)
# image_only_model.summary()

In [ ]:
history_image = image_only_model.fit(x=[X_Train_bulk_img_train,X_Train_inter_img_train], 
                                     y = Y_Train_Hg_porosity_train, batch_size=20,validation_split=0.15, 
                                     epochs=25,verbose=1)

In [ ]:
y_pred = image_only_model.predict([X_Train_bulk_img_holdout,X_Train_inter_img_hodlout]) 

In [ ]:
for y_pred_i, y_true_i in zip(y_pred, Y_Train_Hg_porosity_holdout):
    plt.plot(y_pred_i, label="pred")
    plt.plot(y_true_i,label="True")
    plt.legend()
    plt.show()

In [ ]:
plt.plot(history_image.history["loss"],label="train")
plt.plot(history_image.history["val_loss"],label="test")
plt.legend()
plt.yscale('log')

In [ ]:
np.mean(history_image.history["val_loss"])

# Slice Images + Interfaces+ Extracted Porosity Training set = Total Model

In [ ]:
def build_image_plus_porosity_model():
    model1 = Sequential()
    model1.add(InputLayer(input_shape=(224,224), name= "bulk_img_input"))
    model1.add(Reshape((224,224,1)))
    model1.add(Convolution2D(16,5,strides=(3,3), padding="valid", activation="relu"))
    model1.add(Convolution2D(16,5,strides=(3,3), padding="valid", activation="relu"))
    
    model1.add(Convolution2D(16,5,strides=(3,3), padding="valid", activation="relu"))
    model1.add(Flatten())

    model2 = Sequential()
    model2.add(InputLayer(input_shape=(224, 224), name="interface_img_input"))
    model2.add(Reshape((224,224,1)))
    model2.add(Convolution2D(16,5,strides=(3,3), padding="valid", activation="relu"))
    model2.add(Convolution2D(16,5,strides=(3,3), padding="valid", activation="relu"))
    model2.add(Convolution2D(16,5,strides=(3,3), padding="valid", activation="relu"))
    model2.add(Flatten())

    model3 = Sequential()
    model3.add(InputLayer(input_shape=(3,), name= "porosity_input"))
    model3.add(Dense(10, activation="relu"))
    
    im_concat_layer = Concatenate()([model1.output, model2.output])
    dense_por = Dense(5, activation="tanh")(model3.output)
    dense_im = Dense(5, activation="tanh")(im_concat_layer)
    
    concat_layer = Concatenate(name= "Hg_porosity_diff")([dense_por, dense_im])
    
    lambda_layer = Lambda(Cumulative_Sum, name= "Hg_porosity")(concat_layer)
    
    big_model = Model([model1.input, model2.input, model3.input],[lambda_layer])    
    big_model.compile("adam",loss="mean_squared_error")
    
    return big_model

In [ ]:
total_model = build_image_plus_porosity_model()
total_model.summary()

In [ ]:
history_total = total_model.fit(x=[X_Train_bulk_img_train,X_Train_inter_img_train, X_Train_porosity_train], 
                        y = Y_Train_Hg_porosity_train, batch_size=25,validation_split=0.15, 
                        epochs=50,verbose=1)

In [ ]:
y_pred = total_model.predict([X_Train_bulk_img_holdout, X_Train_inter_img_hodlout, 
                              X_Train_porosity_holdout])    

In [ ]:
for y_pred_i, y_true_i in zip(y_pred, Y_Train_Hg_porosity_holdout):
    plt.plot(y_pred_i, label="pred")
    plt.plot(y_true_i,label="True")
    plt.legend()
    plt.show()

In [ ]:
plt.plot(history_total.history["loss"],label="train")
plt.plot(history_total.history["val_loss"],label="test")
plt.legend()
plt.yscale('log')

In [ ]:
np.mean(history_total.history["val_loss"])

# Slice Images + Interface Images ONLY  = Image Only Model

In [ ]:
def build_image_only_model(nfilter = 16, filter_size = 3):
    model = Sequential()
    model.add(InputLayer(input_shape=(224,224), name="bulk_img_input"))
    model.add(Reshape((224,224,1)))
    model.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Flatten())

    model2 = Sequential()
    model2.add(InputLayer(input_shape=(224, 224), name="interface_img_input"))
    model2.add(Reshape((224,224,1)))
    model2.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model2.add(BatchNormalization())
    model2.add(Activation("relu"))
    model2.add(MaxPool2D())
    
    model2.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model2.add(BatchNormalization())
    model2.add(Activation("relu"))
    model2.add(MaxPool2D())
    
    model2.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model2.add(BatchNormalization())
    model2.add(Activation("relu"))
    model2.add(MaxPool2D())
    
    model2.add(Flatten())
   
    concat_layer = Concatenate()([model.output,model2.output])

    dense_1 = Dense(100, activation="tanh")(concat_layer)
    dense_2 = Dense(10, activation="relu",name= "Hg_porosity_diff")(dense_1)
    lambda_layer = Lambda(Cumulative_Sum,name= "Hg_porosity")(dense_2)    
    big_model = Model([model.input, model2.input],[lambda_layer])
    #big_model = Model([model.input, model2.input],[dense_2])

    big_model.compile("adam",loss="mean_squared_error")
    return big_model

In [ ]:
image_only_model = build_image_only_model(nfilter = 4, filter_size=3)
# image_only_model.summary()

In [ ]:
X_Train_bulk_img_train.shape

In [ ]:
history_image = image_only_model.fit(x=[X_Train_bulk_img_train,X_Train_inter_img_train], 
                                     y = Y_Train_Hg_porosity_train, batch_size=20,validation_split=0.15, 
                                     epochs=25,verbose=1)

In [ ]:
y_pred = image_only_model.predict([X_Train_bulk_img_holdout,X_Train_inter_img_hodlout]) 

In [ ]:
for y_pred_i, y_true_i in zip(y_pred, Y_Train_Hg_porosity_holdout):
    plt.plot(y_pred_i, label="pred")
    plt.plot(y_true_i,label="True")
    plt.legend()
    plt.show()

In [ ]:
plt.plot(history_image.history["loss"],label="train")
plt.plot(history_image.history["val_loss"],label="test")
plt.legend()
plt.yscale('log')

In [ ]:
np.mean(history_image.history["val_loss"])

# Extracted Porosity ONLY Training  = Porosity Only Model

In [ ]:
def build_porosity_ony_model():    

    model3 = Sequential()
    model3.add(InputLayer(input_shape=(3,), name="porosity_input"))
    model3.add(Dense(100, activation="relu"))

    dense_1 = Dense(100, activation="tanh")( model3.output)
    dense_2 = Dense(10, activation="relu",name= "Hg_porosity_diff")(dense_1)
    lambda_layer = Lambda(Cumulative_Sum,name= "Hg_porosity")(dense_2)
    
    big_model = Model(model3.input,[lambda_layer])    
    big_model.compile("adam",loss="mean_squared_error")
    return big_model

In [ ]:
porosity_only_model = build_porosity_ony_model()
# porosity_only_model.summary()

In [ ]:
history_porosity = porosity_only_model.fit(x=X_Train_porosity_train, y = Y_Train_Hg_porosity_train, 
                                           batch_size=62,validation_split=0.15, epochs=100,verbose=2)

In [ ]:
y_pred = porosity_only_model.predict(X_Train_porosity_holdout) 

In [ ]:
for y_pred_i, y_true_i in zip(y_pred, Y_Train_Hg_porosity_holdout):
    plt.plot(y_pred_i, label="pred")
    plt.plot(y_true_i,label="True")
    plt.legend()
    plt.show()

In [ ]:
plt.plot(history_porosity.history["loss"],label="train")
plt.plot(history_porosity.history["val_loss"],label="test")
plt.legend()
plt.yscale('log')

# Transfer Learning from VGG19

In [ ]:
from keras.applications.vgg19 import VGG19

In [ ]:
vgg19 = VGG19(weights='imagenet',include_top=False)

In [ ]:
vgg19.trainable = False
X_Train_bulk_rgb_img_train = np.stack([X_Train_bulk_img_train.reshape(-1,224,224)]*3,axis=-1)
X_Train_bulk_rgb_img_holdout = np.stack([X_Train_bulk_img_holdout.reshape(-1,224,224)]*3,axis=-1)

In [ ]:
X_Train_bulk_VGG19_train = vgg19.predict(X_Train_bulk_rgb_img_train)
X_Train_bulk_VGG19_holdout = vgg19.predict(X_Train_bulk_rgb_img_holdout)

In [ ]:
np.shape(X_Train_bulk_VGG19_train)

In [ ]:
def build_vgg19_bulk_only_model(nfilter = 16, filter_size = 2):
    model = Sequential()
    model.add(InputLayer(input_shape=(7,7,512), name="vgg19_img_input"))
    model.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))    
    model.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))

    model.add(Flatten())   

    dense_1 = Dense(16, activation="tanh")(model.output)
    dense_2 = Dense(10, activation="relu",name= "Hg_porosity_diff")(dense_1)
    lambda_layer = Lambda(Cumulative_Sum,name= "Hg_porosity")(dense_2)    
    big_model = Model(model.input,lambda_layer)    

    big_model.compile("adam",loss="mean_squared_error")
    return big_model

In [ ]:
vgg_bulk_only_model = build_vgg19_bulk_only_model(nfilter=8)

In [ ]:
vgg_bulk_only_model_histoy = vgg_bulk_only_model.fit(X_Train_bulk_VGG19_train, y = Y_Train_Hg_porosity_train, 
                        batch_size=25,validation_split=0.15, epochs=50,verbose=1)

In [ ]:
y_vgg19_pred_holdout = vgg_bulk_only_model.predict([X_Train_bulk_VGG19_holdout])    

In [ ]:
for y_pred_i, y_true_i in zip(y_vgg19_pred_holdout, Y_Train_Hg_porosity_holdout):
    plt.plot(y_pred_i, label="pred")
    plt.plot(y_true_i,label="True")
    plt.legend()
    plt.show()

In [ ]:
plt.plot(vgg_bulk_only_model_histoy.history["loss"],label="train")
plt.plot(vgg_bulk_only_model_histoy.history["val_loss"],label="test")
plt.legend()
plt.yscale('log')

In [ ]:
filter1 = df_total["Sample"]==df_files["Sample"]

In [ ]:
#Image Alone Models
#500X Images Alone based on ADAM Optimisation and VGG16 ARchitecture
def build_image_only_model_500X(nfilter = 16, filter_size = 3):
    model = Sequential()
    model.add(InputLayer(input_shape=(224,224), name="bulk_img_input"))
    model.add(Reshape((224,224,1)))
    model.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Convolution2D(nfilter,filter_size,strides=(filter_size,filter_size), padding="valid"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Flatten())
    
    dense_1 = Dense(100, activation="tanh")(model.output)
    dense_2 = Dense(10, activation="relu", name= "Hg_porosity_diff")(dense_1)
    lambda_layer = Lambda(Cumulative_Sum,name= "Hg_porosity")(dense_2)    
    big_model = Model([model.input],[lambda_layer])
    #big_model = Model([model.input, model2.input],[dense_2])

    big_model.compile("adam",loss="mean_squared_error")
    return big_model

In [ ]:
image_only_model_500X = build_image_only_model_500X()
image_only_model_500X.summary()

In [ ]:
display(df_total.head())

In [ ]:
display(df_files.head())

In [ ]:
#Roughness Parameter Models
#Roughness Scalar Parameter Alone

def build_roughness_scalar_model():    
    model3 = Sequential()
    model3.add(InputLayer(input_shape=(3,), name="roughness_input"))
    model3.add(Dense(100, activation="relu"))

    dense_1 = Dense(100, activation="tanh")( model3.output)
    dense_2 = Dense(10, activation="relu",name= "Hg_porosity_diff")(dense_1)
    lambda_layer = Lambda(Cumulative_Sum,name= "Hg_porosity")(dense_2)
    
    big_model = Model(model3.input,[lambda_layer])    
    big_model.compile("adam",loss="mean_squared_error")
    return big_model

In [ ]:
roughness_scalar_model = build_roughness_scalar_model()
roughness_scalar_model.summary()

In [ ]:
df_roughness

In [ ]:
Hg_df = Hg_porosity_df.drop_duplicates(subset=['Sample'])
Hg_df.reset_index(drop=True, inplace=True)
Hg_df.drop(columns=['Image_ID', 'Slice_ID'], inplace=True)

In [ ]:
display(Hg_df)

In [ ]:
X_Train_roughness_scalar_train = df_roughness[["Ra_(microns)","Rq_(microns)","Rz_(microns)"]].values

In [ ]:
history_roughness_scalar = roughness_scalar_model.fit(x=X_Train_roughness_scalar_train, 
                                           y = Y_Train_Hg_porosity_train, batch_size=62,validation_split=0.15, epochs=100,verbose=2)